In [1]:
#grid_walker.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#3 February 2018

In [2]:
#import plotting libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, font='DejaVu Sans')

In [ ]:
import numpy as np
